In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qcsapphire
import pprint
import time
pp = pprint.PrettyPrinter(indent=4)

In [3]:
qcsapphire.discover_devices()

[['COM3',
  'Intel(R) Active Management Technology - SOL (COM3)',
  'PCI\\VEN_8086&DEV_43E3&SUBSYS_0A541028&REV_11\\3&11583659&1&B3'],
 ['COM5',
  'USB Serial Device (COM5)',
  'USB VID:PID=0483:A3E5 SER=206A36705430 LOCATION=1-8:x.0'],
 ['COM10',
  'USB Serial Device (COM10)',
  'USB VID:PID=04D8:000A SER= LOCATION=1-2:x.0']]

In [4]:
p = qcsapphire.Pulser('COM10')

In [5]:
p.report_global_settings()

[(':PULSE0:STATE?', '1'),
 (':PULSE0:PERIOD?', '0.000050000'),
 (':PULSE0:MODE?', 'NORM'),
 (':PULSE0:BCOUNTER?', '5'),
 (':PULSE0:PCOUNTER?', '3'),
 (':PULSE0:OCOUNTER?', '2'),
 (':PULSE0:EXTERNAL:MODE?', 'DIS'),
 (':PULSE0:EXTERNAL:LEVEL?', '2.50'),
 (':PULSE0:EXTERNAL:EDGE?', 'RIS'),
 (':PULSE0:EXTERNAL:POLARITY?', 'HIGH')]

In [32]:
print('global settings')
pp.pprint(p.report_global_settings())

for channel in range(1,5):
    pp.pprint(f'channel {channel}')
    pp.pprint(p.report_channel_settings(channel))

global settings
[   (':PULSE0:STATE?', '1'),
    (':PULSE0:PERIOD?', '0.000050000'),
    (':PULSE0:MODE?', 'NORM'),
    (':PULSE0:BCOUNTER?', '5'),
    (':PULSE0:PCOUNTER?', '3'),
    (':PULSE0:OCOUNTER?', '2'),
    (':PULSE0:EXTERNAL:MODE?', 'DIS'),
    (':PULSE0:EXTERNAL:LEVEL?', '2.50'),
    (':PULSE0:EXTERNAL:EDGE?', 'RIS'),
    (':PULSE0:EXTERNAL:POLARITY?', 'HIGH')]
'channel 1'
[   (':PULSE1:STATE?', '1'),
    (':PULSE1:WIDTH?', '0.000001000'),
    (':PULSE1:DELAY?', '0.00000600'),
    (':PULSE1:SYNC?', 'T0'),
    (':PULSE1:MUX?', '1'),
    (':PULSE1:POLARITY?', 'NORM'),
    (':PULSE1:OUTPUT:AMPLITUDE?', '5.00'),
    (':PULSE1:CMODE?', 'NORM'),
    (':PULSE1:BCOUNTER?', '5'),
    (':PULSE1:PCOUNTER?', '4'),
    (':PULSE1:WCOUNTER?', '0'),
    (':PULSE1:CGATE?', 'DIS')]
'channel 2'
[   (':PULSE2:STATE?', '0'),
    (':PULSE2:WIDTH?', '0.000001000'),
    (':PULSE2:DELAY?', '0.00000000'),
    (':PULSE2:SYNC?', 'T0'),
    (':PULSE2:MUX?', '2'),
    (':PULSE2:POLARITY?', 'NORM'),
    (

In [10]:
p.system.mode('normal')

'ok'

In [26]:
## set up continuous pulses on channel c
p.pulse0.mode('normal')

period = 50e-6 
width_laser = 5e-6
p.pulse0.period(period) 
p.pulse0.external.mode('disabled')
p.pulse3.cmode('normal')
p.pulse3.polarity('normal')
p.pulse3.width(width_laser) #500 ms wide pulse

#don't turn on yet
# p.pulse3.state(0)
# p.pulse0.state(0)

'ok'

In [8]:
p.set_all_state_off()

In [30]:
p.pulse3.mux(4)
p.pulse4.mux(4) #channel D shows channel C timer output

'ok'

In [23]:
#turn on
p.pulse0.state(1)
p.pulse3.state(1)
p.pulse4.state(1)

'ok'

In [14]:
p.pulse3.output.amplitude(5)
p.pulse4.output.amplitude(5)

'ok'

In [54]:
aom_delay = 1.2e-6

In [55]:
## set up for software trigger pulses on channel a
#p.pulse0.mode('single')
#p.pulse0.period(1.0) #1s period
#p.pulse0.external.mode('trigger')
p.pulse1.cmode('normal')
p.pulse1.polarity('normal')
width_rf = 1e-6
p.pulse1.width(width_rf) #100 ms wide pulse
p.pulse1.delay(width_laser + aom_delay)
p.pulse1.state(1)

'ok'

In [67]:
rf_delay = 0e-6

In [68]:
#configure  channel B
p.pulse2.cmode('normal')
p.pulse2.polarity('normal')
width_laser_readout = 5e-6
p.pulse2.width(width_laser_readout) #100 ms wide pulse
p.pulse2.delay(width_laser +  aom_delay + width_rf + rf_delay)
p.pulse2.state(1)


'ok'

In [60]:
p.pulse3.mux(4 + 2) #set channel C to show output of C & B

'ok'

In [59]:
p.pulse4.mux(4+2) #set channel D to show output of C & B
p.pulse4.output.amplitude(3.3)

'ok'

In [61]:
#wait N seconds between triggers
wait_N = 5.0
p.pulse0.state(1)
p.pulse1.state(1)

for i in range(50):
    time.sleep(wait_N)
    p.software_trigger()

p.pulse0.state(0)
p.pulse1.state(0)

KeyboardInterrupt: 

In [87]:
p.pulse0.state(1)
p.pulse1.state(1)

'ok'

In [88]:
p.software_trigger()

'ok'

In [89]:
p.close()